In [1]:
import pandas as pd 

In [3]:
filename = '../tmp/fitbit.jsonl'
df = pd.read_json(filename, lines=True)
df.head()

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,NaN,None,2017-08-19 21:37:16,"{u'symbols': [], u'user_mentions': [], u'hasht...",NaN,0,False,None,899022480144162816,899022480144162816,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Win a Fitbit Charge 2! https://t.co/FjbEa5unap...,False,"{u'follow_request_sent': False, u'has_extended..."
1,NaN,None,2017-08-19 21:36:45,"{u'symbols': [], u'user_mentions': [{u'indices...",NaN,0,False,None,899022347742711813,899022347742711808,...,NaN,NaN,NaN,9,False,"{u'contributors': None, u'truncated': True, u'...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @realbuzzcom: Take your #training to the ne...,False,"{u'follow_request_sent': False, u'has_extended..."
2,NaN,None,2017-08-19 21:36:14,"{u'symbols': [], u'user_mentions': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,0,False,None,899022218830729217,899022218830729216,...,NaN,NaN,NaN,0,False,NaN,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",Fitbit Blaze + Stainless steel and comfy silic...,False,"{u'follow_request_sent': False, u'has_extended..."
3,NaN,None,2017-08-19 21:36:01,"{u'symbols': [], u'user_mentions': [], u'hasht...",{u'media': [{u'expanded_url': u'https://twitte...,0,False,None,899022165009473540,899022165009473536,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",Check out my trophy for getting first place in...,False,"{u'follow_request_sent': False, u'has_extended..."
4,NaN,None,2017-08-19 21:35:58,"{u'symbols': [], u'user_mentions': [{u'indices...",NaN,0,False,None,899022151566708737,899022151566708736,...,NaN,NaN,NaN,3,False,"{u'contributors': None, u'truncated': False, u...","<a href=""http://twitter.com/download/android"" ...",RT @kevinscampoli: My Fitbit tracks jerking of...,False,"{u'follow_request_sent': False, u'has_extended..."


In [24]:
df.iloc[0].text

u'Win a Fitbit Charge 2! https://t.co/FjbEa5unap help me win!!! #contest #win'

In [29]:
df.count()

contributors                    0
coordinates                     2
created_at                   1000
entities                     1000
extended_entities             197
favorite_count               1000
favorited                    1000
geo                             2
id                           1000
id_str                       1000
in_reply_to_screen_name       141
in_reply_to_status_id         110
in_reply_to_status_id_str     110
in_reply_to_user_id           141
in_reply_to_user_id_str       141
is_quote_status              1000
lang                         1000
metadata                     1000
place                          24
possibly_sensitive            580
quoted_status                   8
quoted_status_id               10
quoted_status_id_str           10
retweet_count                1000
retweeted                    1000
retweeted_status              200
source                       1000
text                         1000
truncated                    1000
user          

In [36]:
consumer_df = df[df['text'].apply(lambda x: 'https' not in x)]

In [46]:
consumer_df.iloc[4].text

u"RT @AlastairCallum: Got bored, here's a #unity3d #webgl app for visualising your #Fitbit data with @Mapbox and @fitbit dev API #screenshots\u2026"

In [49]:
consumer_df.text.iloc[4]

u"RT @AlastairCallum: Got bored, here's a #unity3d #webgl app for visualising your #Fitbit data with @Mapbox and @fitbit dev API #screenshots\u2026"

In [52]:
df.iloc[5]

contributors                                                               NaN
coordinates                                                               None
created_at                                                 2017-08-19 21:35:51
entities                     {u'symbols': [], u'user_mentions': [{u'indices...
extended_entities                                                          NaN
favorite_count                                                               0
favorited                                                                False
geo                                                                       None
id                                                          899022120436584448
id_str                                                      899022120436584448
in_reply_to_screen_name                                        RichardPMacLean
in_reply_to_status_id                                              8.98946e+17
in_reply_to_status_id_str                           

In [21]:
twts = pd.read_csv('data/Tweets.csv')
twts.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [23]:
sanders = pd.read_csv('sander')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
tweet_id                        14640 non-null int64
airline_sentiment               14640 non-null object
airline_sentiment_confidence    14640 non-null float64
negativereason                  9178 non-null object
negativereason_confidence       10522 non-null float64
airline                         14640 non-null object
airline_sentiment_gold          40 non-null object
name                            14640 non-null object
negativereason_gold             32 non-null object
retweet_count                   14640 non-null int64
text                            14640 non-null object
tweet_coord                     1019 non-null object
tweet_created                   14640 non-null object
tweet_location                  9907 non-null object
user_timezone                   9820 non-null object
dtypes: float64(2), int64(2), object(11)
memory usage: 1.7+ MB
